### Cine21 web scraping
* url : http://www.cine21.com/rank/person/
* In developer tool, xhr >> content (Checking request method)
* header check... \>\>
* Request Method : POST
* Form Data(One month data)
    - section : actor
    - period_start : 2020-07
    - gender : all
    - page : 1
* Scarp one or two years data, store that to actors.json

In [224]:
import requests
import json, re
from bs4 import BeautifulSoup
import pymongo
from urllib.parse import urljoin

In [225]:
actor_url = "http://www.cine21.com/rank/person/content"
form_data = {
    'section' : 'actor',
    'period_start' : '2020-07',
    'gender' : 'all',
    'page' : 2
}
res = requests.post(url = actor_url, data = form_data)
soup = BeautifulSoup(res.text, 'html.parser')

hit = soup.select('ul.num_info > li > strong')
mov_list = soup.select('ul.mov_list')
ranks = soup.select('.grade')

actor = []
## this is novel (enumerate and index)
for index, name in enumerate(soup.select('div.name')): 
    # for a in name.select('a'):
    href = name.a['href']
    name = re.sub(r'\(\w*\)', '' ,name.text)
    movie_titles = mov_list[index].select('li a span') ## this is novel
    rank = ranks[index].text
    m_l = []
    for mt in movie_titles:
        m_l.append(mt.text)
    actor.append({
        'name' : name, 
        'href': href, 
        'hit' : int(hit[index].text.replace(',','')), ## this is novel
        'movie-list' : m_l,
        'rank' : int(rank),
        })

#### REGEX
Greedy(.\*) vs Non-Greedy(.\*?)
* Greedy : Select all tags
* Non-Greedy : Select first match
* \[\n\t\r\] : Select one of them at least
* re.complie(r'\[\n\t\r\]) : r'...'(regular expression) > create complier that select with that RE
    - in complie(), expression should be followed after r''

In [226]:
actor_items = []
for nh in actor:

    actor_item = {}
    actor_item['이름'] = nh['name']
    actor_item['흥행지수'] = nh['hit']
    actor_item['출연영화'] = nh['movie-list']
    actor_item['랭킹'] = nh['rank']

    url = urljoin(actor_url,nh['href'])
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    for li in soup.select('.default_info li'):
        actor_item_key = li.select_one('span.tit').text
        # print(actor_item_key)
        actor_item_value = re.sub('<span.*?>.*?</span>|<.*?>','',str(li))
        regex = re.compile(r'[\n\r\t]')
        actor_item_value= regex.sub('',actor_item_value)
        actor_item[actor_item_key] = actor_item_value

    actor_items.append(actor_item)
import pprint
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(actor_items)        
        # print(actor_item_value)



[   {   '랭킹': 8,
        '생년월일': '1990-02-18',
        '성별': '여',
        '소속사': '아바 엔터테인먼트',
        '신장/체중': '165cm, 45kg',
        '이름': '박신혜',
        '직업': '배우',
        '출연영화': ['＃살아있다'],
        '취미': '음악감상',
        '특기': '댄스',
        '학교': '중앙대학교 연극영화학 (재학)',
        '홈페이지': 'https://twitter.com/ssinzhttps://www.instagram.com/ssinz7/',
        '흥행지수': 43667},
    {   '다른 이름': '김주년',
        '랭킹': 9,
        '생년월일': '1981-08-07',
        '성별': '남',
        '이름': '김도윤',
        '직업': '배우',
        '출연영화': ['반도'],
        '흥행지수': 39916},
    {   '랭킹': 10,
        '생년월일': '1970-06-02',
        '성별': '남',
        '이름': '전배수',
        '직업': '배우',
        '출연영화': ['＃살아있다'],
        '흥행지수': 32750},
    {   '랭킹': 11,
        '생년월일': '1985-06-17',
        '성별': '남',
        '신장/체중': '180cm, 68kg',
        '이름': '이현욱',
        '직업': '배우',
        '출연영화': ['＃살아있다'],
        '흥행지수': 21833},
    {   '랭킹': 12,
        '성별': '여',
        '이름': '오혜원',
        '직업': '모델',
        '출연영화': ['＃살아